In [2]:
import Levenshtein

In [3]:
def Edit_distance_str(str1,str2):
    edit_distance = Levenshtein.distance(str1,str2)
    similarity = 1-(edit_distance/max(len(str1),len(str2)))
    return {'distance':edit_distance,'similarity':similarity}

In [4]:
a = Edit_distance_str('搜易贷','搜易贷（北京）金融信息服务有限公司')

In [19]:
a

{'distance': 14, 'similarity': 0.17647058823529416}

In [17]:
b = Edit_distance_str('搜易贷','青岛搜易贷经济信息咨询有限公司')

In [18]:
b

{'distance': 12, 'similarity': 0.19999999999999996}

In [10]:
'''
计算相似度，前提是：将文本转化为向量
使用jieba分词，词袋模型，tf-idf模型，LSI模型 
'''

'\n计算相似度，前提是：将文本转化为向量\n使用jieba分词，词袋模型，tf-idf模型\n'

In [7]:
import jieba.posseg as pseg
import codecs
from gensim import corpora,models,similarities

In [9]:
#构建停用词表
def stopwords_list(filepath):
    stop_words_file = filepath
    stopwords = codecs.open(stop_words_file,'r',encoding='utf8').readlines()
    stopwords_list = [words.strip() for words in stopwords]
    stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
    return stopwords_list,stop_flag

#对一篇文章，或者句子分词、去停用词
def tokenization(filename):
    res = [] 
    with open(filename,'r') as f:
        text = f.read()
        words = pseg.cut(text)
    stopwordslist,stop_flag = stopwords_list('/Users/taoxudong/huli/test/stop_words.txt')
    #分词特性？中文对应着stop_flag的含义。？？？？？？
    for word,flag in words:
        if word not in stopwordslist and flag not in stop_flag:
            if word != '\t':
                res.append(word)
    return res

#加载要处理的文本，进行分词
filenames = ['/Users/taoxudong/huli/test/测试数据二.txt',
            '/Users/taoxudong/huli/test/测试数据三.txt',
            '/Users/taoxudong/huli/test/测试数据四.txt']

corpus = []
for each in filenames:
    corpus.append(tokenization(each))
#建立词袋模型，生成字典和向量语料
print(corpus)
dictionary = corpora.Dictionary(corpus)
doc_vectors = [dictionary.doc2bow(text) for text in corpus]

[['搜易', '贷', '北京', '金融信息', '服务', '有限公司'], ['北京', '搜狐', '互联网', '信息', '服务', '有限公司'], ['青岛', '搜易', '贷', '经济', '信息', '咨询', '有限公司']]


In [15]:
#建立tf-idf模型
tfidf = models.TfidfModel(doc_vectors)
tfidf_vectors = tfidf[doc_vectors]
#len(tfidf_vectors) = 文本数  len(tfidf_vectors[0] == 第一个文本中的词数量)
print(len(tfidf_vectors))
query = tokenization('/Users/taoxudong/huli/test/测试数据一.txt')
print(query)
query_bow = dictionary.doc2bow(query)
index = similarities.MatrixSimilarity(tfidf_vectors)
sims = index[query_bow]
print(list(enumerate(sims)))

3
['搜易', '贷']
[(0, 0.41993365), (1, 0.0), (2, 0.28270504)]


In [12]:
#构建LSI模型，设置主题数（类数）
lsi = models.LsiModel(tfidf_vectors,id2word = dictionary,num_topics=2)
lsi_vector = lsi[tfidf_vectors]
#LSI向量空间中，所有文本的向量都是二维的
query = tokenization('/Users/taoxudong/huli/test/测试数据一.txt')
query_bow = dictionary.doc2bow(query)
query_lsi = lsi[query_bow]
print(query)
index = similarities.MatrixSimilarity(lsi_vector)
sims = index[query_lsi]
print(list(enumerate(sims)))

['搜易', '贷']
[(0, 0.84972703), (1, 0.29139632), (2, 0.89929235)]
